# Create a Project

In [1]:
from morpheus.modflow.types.User import UserId
from morpheus.modflow.types.Project import ProjectId, Name, Description, Tags

user_id = UserId.new()
project_id = ProjectId.new()
name = Name.from_str('Project 1')
description = Description.from_str('Description 1')
tags = Tags.from_list(['tag1', 'tag2'])

from morpheus.modflow.application.write.ProjectCommandHandlers import CreateProjectCommand, CreateProjectCommandHandler

create_project_command = CreateProjectCommand(
  project_id=project_id,
  name=name,
  description=description,
  tags=tags,
  created_by=user_id
)

CreateProjectCommandHandler.handle(create_project_command)

# Read ProjectList

In [2]:
from morpheus.modflow.application.read.ProjectsReader import ProjectsReader

projects_reader = ProjectsReader()
project_list = projects_reader.get_project_summaries()
project_list

[ProjectSummary(project_id=ProjectId(value='0dc6b307-8de3-4030-aed4-7e7ad46c4411'), project_name=Name(value='Project 1'), project_description=Description(value='Description 1'), project_tags=Tags(value=['tag1', 'tag2']), owner_id=UserId(value='10abf654-d37f-4d11-b4f8-6ec04990af46'), visibility=<Visibility.PRIVATE: 'private'>),
 ProjectSummary(project_id=ProjectId(value='edc0f642-88e8-455e-934b-b35c59b6a4dc'), project_name=Name(value='Project 1'), project_description=Description(value='Description 1'), project_tags=Tags(value=['tag1', 'tag2']), owner_id=UserId(value='ef54d174-e79b-47cd-a353-7f47635158da'), visibility=<Visibility.PRIVATE: 'private'>)]

# Create base model

In [3]:
from morpheus.modflow.types.discretization.spatial import Rotation, LengthUnit
from morpheus.modflow.types.geometry import Polygon
from morpheus.modflow.types.ModflowModel import ModelId
from morpheus.modflow.application.write.BaseModelCommandHandlers import CreateBaseModelCommand, CreateBaseModelCommandHandler

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

nx = 100
ny = 50
rotation = Rotation.from_float(0.0)
length_unit = LengthUnit.meters()

create_base_model_command = CreateBaseModelCommand(
  project_id=project_id,
  model_id=ModelId.new(),
  geometry=polygon,
  grid_properties={
    'nx': nx,
    'ny': ny,
    'rotation': rotation.to_float(),
    'length_unit': length_unit.to_str()
  },
  created_by=user_id
)

CreateBaseModelCommandHandler.handle(create_base_model_command)

# Read Latest BaseModel

In [4]:
from morpheus.modflow.application.read.BaseModelReader import BaseModelReader

base_model_reader = BaseModelReader()
base_model = base_model_reader.get_latest_base_model(project_id)
base_model

ModflowModel(model_id=ModelId(value='2bddd492-e22a-423d-a205-a96c573eef43'), spatial_discretization=SpatialDiscretization(geometry=Polygon(coordinates=[[[13.922514437551428, 50.964720483303836], [13.925250781947113, 50.965228748412386], [13.925036413951403, 50.96623732041704], [13.92222441026388, 50.96629040370362], [13.922514437551428, 50.964720483303836]]], type='Polygon'), grid=Grid(origin=Point(coordinates=[13.922224410263878, 50.9662904037036], type='Point'), x_distances=[3.3689415472885593, 3.3689415472885593, 3.3689415472885598, 3.368941547288559, 3.368941547288559, 3.3689415472885607, 3.3689415472885607, 3.368941547288557, 3.368941547288557, 3.3689415472885607, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.368941547288564, 3.36894154728855, 3.368941547288564, 3.368941547288557, 3.368941547288557, 3.3689415472885713, 3.368941547288557, 3.368941547288557, 3.368941547288557, 3.368941547288557, 3.3689415472885713

## Update BaseModel Geometry


In [5]:
from morpheus.modflow.application.write.BaseModelCommandHandlers import UpdateModelGeometryCommandHandler, UpdateModelGeometryCommand

new_polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922, 50.965),
    (13.925, 50.965),
    (13.925, 50.966),
    (13.922, 50.966),
    (13.922, 50.965)
  ]]
)

command = UpdateModelGeometryCommand(
  project_id=project_id,
  geometry=new_polygon,
  updated_by=user_id
)

UpdateModelGeometryCommandHandler.handle(command)

base_model_reader = BaseModelReader()
base_model = base_model_reader.get_latest_base_model(project_id)
base_model.spatial_discretization.geometry

Polygon(coordinates=[[[13.922, 50.965], [13.925, 50.965], [13.925, 50.966], [13.922, 50.966], [13.922, 50.965]]], type='Polygon')

## Update BaseModel Grid Properties

In [6]:
from morpheus.modflow.application.write.BaseModelCommandHandlers import UpdateModelGridCommandHandler, UpdateModelGridCommand

command = UpdateModelGridCommand(
  project_id=project_id,
  update_grid={
    'relative_x_coordinates': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
    'relative_y_coordinates': [0, 0.2, 0.4, 0.6, 0.8, 1],
    'rotation': 10,
  },
  updated_by=user_id
)

UpdateModelGridCommandHandler.handle(command=command)

base_model_reader = BaseModelReader()
base_model = base_model_reader.get_latest_base_model(project_id)
print(base_model.spatial_discretization.geometry)
print(base_model.spatial_discretization.grid)
print(base_model.spatial_discretization.affected_cells.to_dict())

Polygon(coordinates=[[[13.922, 50.965], [13.925, 50.965], [13.925, 50.966], [13.922, 50.966], [13.922, 50.965]]], type='Polygon')
Grid(origin=Point(coordinates=[13.921728464165456, 50.96596984662515], type='Point'), x_distances=[35.957842737529425, 35.957842737529425, 35.95784273752943, 35.95784273752942, 35.95784273752943, 35.95784273752943, 35.957842737529404, 35.95784273752943, 35.95784273752946, 35.957842737529404], y_distances=[46.41259297113866, 46.41259297113866, 46.412592971138636, 46.41259297113868, 46.41259297113865], rotation=Rotation(value=10), length_unit=LengthUnit(unit=2))
{'type': 'raster', 'empty_value': False, 'shape': [5, 10], 'data': [[False, True, True, True, True, False, False, False, False, False], [False, True, True, True, True, True, True, True, True, True], [True, True, True, True, True, True, True, True, True, True], [True, True, True, True, True, True, True, True, True, False], [False, False, False, False, False, True, True, True, True, False]]}


## Update BaseModel Affected Cells

In [7]:
from morpheus.modflow.application.write.BaseModelCommandHandlers import UpdateModelAffectedCellsCommandHandler, UpdateModelAffectedCellsCommand

base_model_reader = BaseModelReader()
base_model = base_model_reader.get_latest_base_model(project_id=project_id)
affected_cells = base_model.spatial_discretization.affected_cells

assert affected_cells.is_active(x=0, y=0) == False
affected_cells.set_active(x=0, y=0)
assert affected_cells.is_active(x=0, y=0) == True

command = UpdateModelAffectedCellsCommand(
  project_id=project_id,
  affected_cells=affected_cells,
  updated_by=user_id
)

UpdateModelAffectedCellsCommandHandler.handle(command=command)

In [8]:
base_model_reader = BaseModelReader()
base_model = base_model_reader.get_latest_base_model(project_id=project_id)
affected_cells = base_model.spatial_discretization.affected_cells
assert affected_cells.is_active(x=0, y=0) == True

## Show Versions of BaseModel so far

In [9]:
base_model_reader = BaseModelReader()
base_model_reader.get_versions(project_id=project_id)

[BaseModelVersion(version_id=VersionId(value='ca4e9283-0d95-4283-b033-b64931f59526'), tag=VersionTag(value='v0.0.0'), description=VersionDescription(value='Initial version'))]

## Tag Version

In [10]:
from morpheus.modflow.types.BaseModelVersion import VersionTag, VersionDescription
from morpheus.modflow.application.write.BaseModelCommandHandlers import SetVersionCommand, SetVersionCommandHandler

command = SetVersionCommand(
  project_id=project_id,
  version_tag=VersionTag('v1.0.0'),
  version_description=VersionDescription('Version 1.0.0'),
  created_by=user_id
)

SetVersionCommandHandler.handle(command=command)

In [11]:
base_model_reader.get_versions(project_id=project_id)

[BaseModelVersion(version_id=VersionId(value='ba129f6d-4dd4-4301-bf5d-58e5ea65107e'), tag=VersionTag(value='v1.0.0'), description=VersionDescription(value='Version 1.0.0')),
 BaseModelVersion(version_id=VersionId(value='ca4e9283-0d95-4283-b033-b64931f59526'), tag=VersionTag(value='v0.0.0'), description=VersionDescription(value='Initial version'))]

In [12]:
version = base_model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version

BaseModelVersion(version_id=VersionId(value='ba129f6d-4dd4-4301-bf5d-58e5ea65107e'), tag=VersionTag(value='v1.0.0'), description=VersionDescription(value='Version 1.0.0'))

In [13]:
from morpheus.modflow.application.write.BaseModelCommandHandlers import UpdateVersionDescriptionCommand, UpdateVersionDescriptionCommandHandler

command = UpdateVersionDescriptionCommand(
  project_id=project_id,
  version_id=version.version_id,
  version_description=VersionDescription('Version 1.0.0 - Updated'),
  updated_by=user_id
)

UpdateVersionDescriptionCommandHandler.handle(command=command)

In [14]:
version = base_model_reader.get_version_by_tag(project_id=project_id, tag=VersionTag('v1.0.0'))
version.description.to_str()

'Version 1.0.0 - Updated'